In [17]:
import pandas as pd
import streamlit as st
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
df = pd.read_csv("./Data/vgsales.csv")
df 

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales;
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74;
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24;
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82;
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33;
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37;
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01;
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01;
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01;
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01;


In [19]:

df_2 = df[['Platform','NA_Sales','Year']]
df_sum = df_2.groupby(['Platform','Year']).sum().reset_index()
df_sum
# a = list(df_sum['NA_Sales'])
# b =list(df_sum['Platform'])
# fig,ax = plt.subplots()

# 
# plt.pie(a, labels=b, autopct='%1.1f%%')
# plt.show()

,Platform,Year,NA_Sales
0,3DS,1994.0,0.00
1,3DS,1995.0,0.00
2,3DS,2011.0,26.98
3,3DS,2012.0,17.04
4,3DS,2013.0,15.33
...,...,...,...
227,XBox,2008.0,0.13
228,XOne,2013.0,11.80
229,XOne,2014.0,30.41
230,XOne,2015.0,34.29


In [20]:
import torch.nn as nn
import torch
import matplotlib.pyplot as plt 
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [21]:
data = pd.read_csv("./IRIS.csv")
data.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [22]:
var_aux = data['species'].dtype
print(var_aux)

object


In [23]:
class_target = input('Digite el nombre de la columna donde estan las clases a predecir: ')
print(class_target)

species


In [24]:
if data[class_target].dtype == 'object':
    encoder = LabelEncoder()
    data["class_target_2"] = encoder.fit_transform(data[class_target]) 
    X= data.drop([class_target,'class_target_2'],axis=1)
    Y = data['class_target_2']
else:
    X= data.drop([class_target],axis=1)
    Y = data[class_target]

In [25]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.long)

In [26]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

In [27]:
input_dim = X.shape[1] 
output_dim = len(Y.unique())
hiden_layers = 10

In [28]:
# class SoftMax(nn.Module):
#     def __init__(self,input_size,output_size):
#         super().__init__()
#         self.linear = nn.Linear(input_size,output_size)
#     def forward(self,x):
#         z = self.linear(x) 
#         return z

In [29]:
Soft_Max = torch.nn.Sequential(
    torch.nn.Linear(input_dim,hiden_layers),
    torch.nn.Sigmoid(),
    torch.nn.Linear(hiden_layers,output_dim),
    torch.nn.Softmax(dim=1))

In [30]:
def train_model(model,n_epochs,learning_rate,train_batch_size,validation_bath_size,train_dataset,validation_dataset):
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=train_batch_size,shuffle=True)
    validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, batch_size=validation_bath_size)
    loss_list = []
    accuracy_list = []
    for epoch in range(n_epochs):
        for x, y in train_loader:
            optimizer.zero_grad()
            z = model(x)
            loss = criterion(z, y)
            loss.backward()
            optimizer.step()
        model.eval()  
        correct = 0
        N_test = len(validation_dataset)  
        with torch.no_grad():
            for x_test, y_test in validation_loader:
                z = model(x_test.view(-1,input_dim))
                _, yhat = torch.max(z.data, 1)
                correct += (yhat == y_test).sum().item()
        accuracy = correct / N_test
        loss_list.append(loss.data)
        accuracy_list.append(accuracy)
    return loss_list, accuracy_list

In [31]:
n_epochs = 100
learning_rate = 0.1
train_batch_size = 100
validation_bath_size = 10
train_dataset = train_dataset
validation_dataset = val_dataset
N_test = len(validation_dataset)
loss_list, accuracy_list = train_model(Soft_Max,n_epochs,learning_rate,train_batch_size,validation_bath_size,train_dataset,validation_dataset)


In [ ]:
x =  torch.tensor([5.9,3.0,5.1,1.8],dtype=torch.float32)
z = Soft_Max(x.reshape(-1,input_dim))
_, yhat = torch.max(z, 1)

if data[class_target].dtype == 'object':
    a = (data[class_target].unique())
    b = (data['class_target_2'].unique())
    c = dict(zip(b,a))
else:
    c = (data[class_target].unique())   
    
# a = (data['species'].unique())
# b = (data['species_2'].unique())3,
# c = dict(zip(b,a))  
for i in range(len(c)):
    if int(yhat[0]) in c:
        print(f'Class predicted with the values enteres is: {c[i]}')
        print(f'clas probability: {_.item()}')
        break

Class predicted with the values enteres is: Iris-setosa
clas probability: 0.5310589671134949
